### 感知机存在的问题
1. 相差一点点，就完全的说成不同的分类，没有明确制定度量标准，例如相差的小概率差距就小，差距就大。
2. sign不可微分，逻辑回归直接跳过了这一步，用力里面的函数进行了梯度下降，此处使用一种可微的函数进行替换。

思想就是结合了感知机和条件概率，构造了一个可微的条件概率公式，对条件概率公式，根据最大似然估计估计参数后，用于其他数据的预测，参数只有两个一个是w,一个是b
#### 极大似然估计计算w，b，导数等于0对应的w和b就是最优解，由于存在求和，为0求解不好实现，因此使用梯度上升法

#### 梯度上升法，求最大值
每一个轮次，对所有的数据执行梯度上升，实现时可将w和b合在一起，x做追加1的操作



In [2]:
# coding: utf-8
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score,confusion_matrix
from tqdm import tqdm
from math import *
from sklearn.decomposition import PCA
import numpy as np
from sklearn.preprocessing import StandardScaler

In [3]:
#文件预处理成tsv格式
def read_file(file_name):
    with open(file_name,'r',encoding='utf-8') as f:
        data = f.readlines()
    label_map = {}
    label_map["ham"] = 0
    label_map["spam"] = 1
    
    
    text = list()
    label = list()
    label_num = list()
    for item in data:
        item = item.strip()
        item = item.split(" ")
        item_len = len(item)
        tmp_list = list()
        if(item[1] in ["ham","spam"]):
            label.append(item[1])
            label_num.append(label_map[item[1]])
            for index in range(2,item_len,2):
                tmp_list.append(item[index])
        text.append(tmp_list)
    return text,label,label_num


train_text,train_label,train_label_num = read_file("../data/train")
test_text,test_label,test_label_num = read_file("../data/test")


train_text = [" ".join(item) for item in train_text]
test_text = [" ".join(item) for item in test_text]



cv = CountVectorizer()
part_fit = cv.fit(train_text) # 以部分句子为参考
train_all_count = cv.transform(train_text) # 对训练集所有邮件统计单词个数
test_all_count = cv.transform(test_text) # 对测试集所有邮件统计单词个数
tfidf = TfidfTransformer()
train_tfidf_matrix = tfidf.fit_transform(train_all_count)
test_tfidf_matrix = tfidf.fit_transform(test_all_count)



print('训练集', train_tfidf_matrix.shape)
print('测试集', test_tfidf_matrix.shape)

#训练数据
train_tfidf_matrix_1 = train_tfidf_matrix.toarray()
#测试数据
test_tfidf_matrix1 = test_tfidf_matrix.toarray()

训练集 (9000, 965)
测试集 (1000, 965)


In [4]:
#进行降维打击
# PCA
pca = PCA(n_components=0.9)# 保证降维后的数据保持90%的信息
pca.fit(train_tfidf_matrix_1)
pca_train_tfidf_matrix_1 = pca.transform(train_tfidf_matrix_1)
pca_train_tfidf_matrix_1.shape
pca_test_tfidf_matrix_1 = pca.transform(test_tfidf_matrix1)
print(pca_train_tfidf_matrix_1.shape)

(9000, 545)


In [38]:
#构建逻辑斯蒂回归模型
#执行一步梯度上升
def get_data(w,x,y):
    wx = (w*x).sum()
    return y*x-x*np.exp(wx)/(1+np.exp(wx))

#根据训练数据获取w
def train_get_parameter(train_data,train_label,iter_num=200):
    m,n = train_data.shape
    #对数据添加一列
    train_data = np.insert(train_data,n,np.ones(m),1)
    #初始化w 
    w = np.zeros(n+1)
    #梯度上升法更新w
    for i in tqdm(range(iter_num)):
        #每一个都执行一下梯度上升，相当于一个batch
        for j in range(m):
            #返回一个n+1维度的数据
            w = w + get_data(w,train_data[j],train_label[j])
    return w
    


#根据w预测每个值的概率
def predict(w,test_data):
    m,n = test_data.shape
    #对test_data进行扩充
    test_data = np.insert(test_data,n,np.ones(m),1)
    result = list()
    #预测
    for i in range(m):
        pro = 1/(1+np.exp((w*test_data[i]).sum()))
        
        if(pro>0.5):
            result.append(0)
        else:
            result.append(1)
    return result


In [ ]:
w = train_get_parameter(pca_train_tfidf_matrix_1,train_label_num)

 38%|██████████████████████████████▍                                                  | 75/200 [00:35<01:11,  1.74it/s]

In [ ]:
result = predict(w,pca_test_tfidf_matrix_1)
acc = accuracy_score(result,test_label_num)
print(acc)